# Main notebook

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import random

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Env Variables

In [3]:
DEFAULT_PLAYER_NB = 2
GRID_SIDE_SIZE = 25
MAX_POSITION = 624 # 25*25-1

## Environment Creation

### Tile

In [4]:
class Tile:
    """
    Description:
        Representation of a game tile. A tile is described by
        its color and its symbol. When a tile is not placed yet,
        it has a default state on the grid: "Blank" color and 
        "Empty" symbol.
        
    Parameters:
        color: (String) Color of the tile. Must be in the valid set of colors.
        
    """
    
    DEFAULT_COLOR = "Blank"
    DEFAULT_SYMBOL = "Empty"
    SET_OF_COLORS = {"Red","Blue","Green","Black","White"}
    SET_OF_SYMBOLS = {"Bird","Dog","Scarab","Scrib","Storage","Desert","Bonus"}
    SYMBOL_NUMBER_DICT = {"Bird":2,"Dog":2,"Scarab":2,"Scrib":1,"Storage":1,"Desert":1,"Bonus":2}
    
    
    def __init__(self,color=DEFAULT_COLOR,symbol=DEFAULT_SYMBOL):
        if color not in Tile.SET_OF_COLORS and color != Tile.DEFAULT_COLOR:
            raise ValueError
        if symbol not in Tile.SET_OF_SYMBOLS and symbol != Tile.DEFAULT_SYMBOL:
            raise ValueError
        self.color = color
        self.symbol = symbol
        
    @staticmethod
    def create_tiles_values_dict():
        """ Create the mapping between integers on the grid and the tiles. """
        tiles_values = {0:Tile(Tile.DEFAULT_COLOR,Tile.DEFAULT_SYMBOL)}
        colors = sorted(Tile.SET_OF_COLORS)
        symbols = sorted(Tile.SET_OF_SYMBOLS)
        i = 1
        for c in colors:
            for s in symbols:
                tiles_values[i] = Tile(c,s)
                i += 1
        return tiles_values

### Deck

In [5]:
class Deck:
    """
    Description:
        Representation of the two stack of Tiles from which the
        players can draw. The stacks are randomly shuffled according
        to the given seed.
        
    Parameters:
        seed: (int) Seed for initialization of the random number generator.
    
    """
    def __init__(self,seed=0):
        random.seed(seed)
        np.random.seed(seed)
        init_deck = []
        for c in Tile.SET_OF_COLORS:
            for s, n in Tile.SYMBOL_NUMBER_DICT.items():
                for i in range(n):
                    init_deck.append(Tile(c,s))
        random.shuffle(init_deck)
        self.stack_0 = init_deck[:len(init_deck)//2]
        self.stack_1 = init_deck[len(init_deck)//2:]
        
    def draw(self,stack):
        """ Draw a tile from the given stack. """
        if stack == 0:
            return self.stack_0.pop(0)
        else:
            return self.stack_1.pop(0)
        
    def tile_available(self):
        """ Return a tuple of the two available tiles'color. """
        Tile_0 = None
        Tile_1 = None
        if len(self.stack_0) != 0:
            Tile_0 = self.stack_0[0].color
        if len(self.stack_1) != 0:
            Tile_1 = self.stack_1[0].color
        return Tile_0, Tile_1

### Token

In [6]:
class Token():
    """
    Description:
        Representation of the token needed to buy
        tiles.
    
    Parameters:
        color: (String) Color of the token. Must be
                        included in "Red","Green","Blue",
                        "Black","White" or "Ankh".
    
    """
    
    REGULAR_COLORS = {"Red","Green","Blue","Black","White"}
    SPECIAL_COLOR = "Ankh"
    
    def __init__(self,color):
        if color not in Token.REGULAR_COLORS and color != Token.SPECIAL_COLOR:
            raise ValueError
        self.color = color

### Ressource Pool

In [7]:
class Ressource_Pool():
    """
    Description:
        Representation of the pool of tokens needed to buy
        tiles. The regular and special colors are treated separately
        as they undergo different rules. Each color pool is a queue
        of tokens.
        
    Parameters:
        init_reg_nb: (int) Initial number of regular token.
        init_special_nb: (int) Initial number of special token.
        max_regular_nb: (int) Maximum number of regular token per color.
        max_special_nb: (int) Maximum number of special token.
        max_tot_reg_nb: (int) Maximum number of regular token accross all colors.
    
    """
    
    def __init__(self,init_reg_nb,init_special_nb,max_regular_nb,
                     max_special_nb,max_tot_reg_nb):
        self.MAX_REGULAR_NB = max_regular_nb
        self.MAX_SPECIAL_NB = max_special_nb
        self.MAX_REG_POOL_SIZE = max_tot_reg_nb
        self.red_tokens = [Token("Red") for i in range(init_reg_nb)]
        self.green_tokens = [Token("Green") for i in range(init_reg_nb)]
        self.blue_tokens = [Token("Blue") for i in range(init_reg_nb)]
        self.black_tokens = [Token("Black") for i in range(init_reg_nb)]
        self.white_tokens = [Token("White") for i in range(init_reg_nb)]
        self.ankh_tokens = [Token("Ankh") for i in range(init_special_nb)]
        self.pool_dict = {"Red":self.red_tokens,"Green":self.green_tokens,
                           "Blue":self.blue_tokens,"Black":self.black_tokens,
                           "White":self.white_tokens,"Ankh":self.ankh_tokens}
        
    def state(self):
        state_dict = dict()
        for c in self.pool_dict.keys():
            state_dict[c] = len(self.pool_dict[c])
        state_dict[Token.SPECIAL_COLOR] = len(self.pool_dict[Token.SPECIAL_COLOR])
        return state_dict
        
    
    def fill(self, color):
        """ Add a token of the given color to the ressource pool. """
        color_pool = self.pool_dict[color]
        max_color_pool_size = (self.MAX_SPECIAL_NB if color == Token.SPECIAL_COLOR
                                    else self.MAX_REGULAR_NB)
        total_regular_pool_size = sum([len(self.pool_dict[c]) for c in Token.REGULAR_COLORS])
        if len(color_pool) >= max_color_pool_size:
            raise Exception(f"Maximum number of token reached for the color {color}")
        elif color != Token.SPECIAL_COLOR and total_regular_pool_size >= self.MAX_REG_POOL_SIZE:
            raise Exception(f"Maximum number of total token reached: {total_regular_pool_size}")
        else:
            color_pool.append(Token(color))
        
    def draw(self, color):
        """ Draw a token of the given color to the ressource pool. """
        if len(self.pool_dict[color]) <= 0:
            raise Exception(f"No token left for the color {color}")
        else:
            self.pool_dict[color].pop(0)

### Shop

In [36]:
class Shop:
    """
    Desciption:
        Representation of the Shop in the game. It possesses
        the deck and the global ressource pool in addition to 
        its standard commodities, namely the prices and the tiles 
        queue. In our implementation, the shop is authoritative. 
        By authoritative, we mean that this is the shop who makes
        the transactions. Thus it has the right to modify the players'
        ressource pools.
        
    Parameters:
        seed: (int) Seed for initialization of the random number generator.
        player_nb: (int) Number of players in the game.
    
    """
    
    def __init__(self,seed=0,player_nb=2):
        # Initialize shop ressource pool
        self.PLAYER_NB = player_nb
        init_token_nb = 3 + self.PLAYER_NB
        self.ressource_pool = Ressource_Pool(init_token_nb,init_token_nb,
                                init_token_nb,init_token_nb,
                                len(Token.REGULAR_COLORS)*init_token_nb)
        # Initialize the game deck
        self.deck = Deck(seed)
        # Randomly shuffle the prices
        random.seed(seed)
        np.random.seed(seed)
        initial_token_pool = [Token(c) for c in list(Token.REGULAR_COLORS)*3]
        random.shuffle(initial_token_pool)
        # Create the fixed list of prices
        self.price_list = []
        j = 0
        for i in [2,2,2,3,3,3]:
            self.price_list.append(initial_token_pool[j:j+i])
            j=j+i
        # Initialize the queue of tiles
        self.SHOP_SIZE = 6
        self.tiles_queue = []
        for i in range(self.SHOP_SIZE):
            stack_choice = np.random.randint(2)
            self.tiles_queue.append(self.deck.draw(stack_choice))
            
    def get_tile_price(self,tile_index):
        """ Return a dictionnary with the amount of token per color. """
        if tile_index < 0 or tile_index >= self.SHOP_SIZE:
            raise ValueError("Invalid tile index.")
        tile_price = dict.fromkeys(self.ressource_pool.pool_dict.keys(),0)
        for c in map(lambda t : t.color,self.price_list[tile_index]):
            tile_price[c] += 1
        return tile_price
    
    def update_tiles_queue(self,tile_index,stack_choice):
        """ Retrieve the wanted tile and update the tile queue. """
        bought_tile = self.tiles_queue.pop(tile_index)
        new_tile = self.deck.draw(stack_choice)
        self.tiles_queue.append(new_tile)
        return bought_tile
            
    def buy(self,tile_index,player_ressource_pool,stack_choice):
        """ Verify if the transaction is valid, and if so perform it. """
        if tile_index < 0 or tile_index >= self.SHOP_SIZE:
            raise ValueError("Invalid tile index.")
        if stack_choice not in [0,1]:
            raise ValueError("Invalid stack choice.")
        tile_price = self.get_tile_price(tile_index)
        player_ressource_state = player_ressource_pool.state()
        # Check if the player has the ressource to buy
        for color, price in tile_price.items():
            if player_ressource_state[color] < price:
                raise Exception(f"Not enough token of color {color} to buy the given tile.")
        # Make the transaction
        for color, price in tile_price.items():
            for i in range(price):
                player_ressource_pool.draw(color)
                self.ressource_pool.fill(color)
        # Update the tile queue
        bought_tile = self.update_tiles_queue(tile_index,stack_choice)
        return bought_tile
            
    def destroy(self,player_ressource_pool,stack_choice):
        """ Destroy the first tile of the queue in exchange of an Ankh. """
        player_ressource_state = player_ressource_pool.state()
        if player_ressource_state[Token.SPECIAL_COLOR] < 1:
            raise Exception(f"Not enough {Token.SPECIAL_COLOR} tokens to destroy.")
        # Make the transaction
        player_ressource_pool.draw(Token.SPECIAL_COLOR)
        self.ressource_pool.fill(Token.SPECIAL_COLOR)
        # Update the tile queue
        destroyed_tile = self.update_tiles_queue(0,stack_choice)
        
    def draw_ressources(self,player_ressource_pool,token_list):
        """ Verifiy if the transaction is valid, and if so perform it. """
        if len(token_list) != 3:
            raise ValueError(f"The token_list should be of lenght 3 not {len(token_list)}.")
        # Reformat tokens list into a dictionnary
        token_colors = [t.color for t in token_list 
                               if (t.color in Token.REGULAR_COLORS or 
                                  t.color == Token.SPECIAL_COLOR)]
        token_dict = dict(self.ressource_pool.pool_dict.keys(),0)
        for c in token_colors:
            token_dict[c] += 1
        # Check if the transaction is valid
        shop_ressource_state = self.ressource_pool.state()
        player_ressource_state = player_ressource_pool.state()
        for color, number in token_dict.items():
            total_regular_pool_size = sum([len(player_ressource_pool.pool_dict[c]) 
                                           for c in Token.REGULAR_COLORS])
            if shop_ressource_state[color] < number:
                raise Exception(f"Not enough token of color {color} in the shop.")
            elif (color == Token.SPECIAL_COLOR and 
                  player_ressource_state[color] >= player_ressource_pool.MAX_SPECIAL_NB):
                raise Exception(f"Already reached maximum number of token of color {color}.")
            elif (color in Token.REGULAR_COLORS and 
                  player_ressource_state[color] >= player_ressource_pool.MAX_REGULAR_NB):
                raise Exception(f"Already reached maximum number of token of color {color}.")
            elif (color in Token.REGULAR_COLORS and 
                  total_regular_pool_size >= player_ressource_pool.MAX_REG_POOL_SIZE):
                raise Exception(f"Already reached maximum number of token for regular colors.")
        # Make the transaction
        for color, number in token_dict.items():
            for i in range(number):
                player_ressource_pool.fill(color)
                self.ressource_pool.draw(color)
        

### Player

In [39]:
class Player:
    """
    Description:
        Representation of a player in the game. It has
        a unique ID and a ressource pool.
        
    Parameters:
        player_id: (int) Unique player id.
    
    """
    
    def __init__(self,player_id):
        self.ressource_pool = Ressource_Pool(0,0,5,2,5) 
        self.ID = player_id

### Global Environment

In [ ]:
class Ankhor_Env:
    def __init__(self,player_nb=DEFAULT_PLAYER_NB):
        self.player_nb = player_nb
        self.grid = np.zeros((GRID_SIDE_SIZE,GRID_SIDE_SIZE,player_nb))
        self.end = False
        self.curr_player = 0
        self.tiles_values = TILES_VALUES_DICT
        self.winner = None
        self.tile_stack = None
        
    def position_coordinates(position):
        """ Retrieve the coordinate on the grid given the integer position. """
        return (position//GRID_SIDE_SIZE,position%GRID_SIDE_SIZE)
    
    def check_basis(self,x,y,player,tile):
        """ Check if the tile can be placed on the relative basis. """
        neighbours = []
        for i in [-1,1]:
            for j in [-1,1]:
                neighbour_tile = self.grid[x+i,y+j,player]
                if neighbour_tile == 0: # Miss a tile in the basis
                    return False
                neighbours.append(neighbour_tile)
        basis_colors = [t.color for t in neighbours]
        return tile.color in basis_colors
                
        
    def check_valid(self,position,player,tile):
        """ Check if the move is valid. """
        if position < 0 or position > MAX_POSITION: # Outside the grid
            return False
        if position % 2 == 1: # Between Tiles
            return False
        pos_x, pos_y = position_coordinates(position)
        if self.grid[pos_x,pos_y,player] != 0: # Tile unavailable
            return False
        if pos_x%2 == 1: # On top of basis
            return check_basis(pos_x,pos_y,player,tile)
        return True

In [ ]:
test = Tile("Red","Bird")